# Jitbit's SQL Interview Questions Practice

### Introduction

Jitbit, a tech company focusing on customer support software, released a list of SQL interview questions to practice on. The database schema is as follows:

![heatmap](../img/table_schema.png)

The questions are as follows:

1. List employees (names) who have a bigger salary than their boss
2. List employees who have the biggest salary in their departments
3. List departments that have less than 3 people in it
4. List all departments along with the number of people there (tricky - people often do an "inner join" leaving out empty departments)
5. List employees that don't have a boss in the same department
6. List all departments along with the total salary there

For easy viewing, all queries will be run in an ipython notebook and displayed with pandas.

### Installation

Run the following in your terminal.

```bash
conda env create -f environment.yml -n sql-query-practice
```

Next, run the bash script download_data to download the data and import it into a Postgres database with this command:

```bash
psql postgres -U psql_username -f db_setup.sql
```
We'll start by importing necessary packages and getting the project file path.

In [1]:
# Loads watermark extension and prints details about current platform
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,psycopg2
 
# import packages
%matplotlib inline
%load_ext dotenv
# Use find_dotenv to locate the file
%dotenv

import pandas as pd
import pandas.io.sql as psql
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import psycopg2

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

Tue Nov 20 2018 

CPython 3.7.0
IPython 7.1.1

numpy 1.15.4
scipy 1.1.0
sklearn 0.20.0
pandas 0.23.4
psycopg2 2.7.5 (dt dec pq3 ext lo64)

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.5.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
/Users/sebp/LocalDocuments2/Data Science/portfolio/SQL/Jitbit_sql_questions


### 0.0 - Initialize Database

The bash script from earlier setupd the database and imported data into a Postgres database. Le's initialize psycopg2 so we can run our queries.

In [2]:
connection = psycopg2.connect(user = os.getenv("DB_USER"),
                                  password = os.getenv("DB_PASSWORD"),
                                  host = os.getenv("DB_HOST"),
                                  port = os.getenv("DB_PORT"),
                                  database = os.getenv("DB_NAME"))
cur = connection.cursor()

Let's take a peak inside to see what our data looks like.


The Departments table.

In [8]:
departments = psql.read_sql("Select * from Departments", connection)
employees = psql.read_sql("Select * from employees", connection)


print ('Departments')
departments.head()


Departments


,departmentid,name
0,1,Exec
1,2,Legal
2,3,IT
3,4,Admin
4,5,Nobody


The Employees Table.

In [9]:
print ('Employees')
employees.head()

Employees


,employeeid,departmentid,bossid,name,salary
0,1,1,1,Chief,100
1,2,3,1,CTO,95
2,3,2,1,CFO,100
3,4,3,2,IT 1,90
4,5,3,2,IT 2,90


# Questions

### 1. List employees (names) who have a bigger salary than their boss.



In [51]:
query = '''
SELECT a.name FROM employees a JOIN employees b
on a.bossid = b.employeeid
WHERE a.salary > b.salary
'''

higher_salaried_employees = psql.read_sql("SELECT a.name FROM employees a JOIN employees b on a.bossid = b.employeeid WHERE a.salary > b.salary", connection)


higher_salaried_employees.head()

,name
0,Adm 2
1,Legal 1
2,IT 6


### 2. List employees who have the biggest salary in their departments


In [59]:
%time

query = '''
SELECT a.employeeid, a.name, a.salary, a.departmentid
FROM employees a JOIN(
    SELECT MAX(a.salary), a.departmentid
    FROM employees a JOIN departments b
    ON a.departmentid = b.departmentid
    GROUP BY a.departmentid) b
on a.salary = b.max AND a.departmentid = b.departmentid
'''


highest_salaried_by_dept = psql.read_sql(query, connection)


highest_salaried_by_dept.head()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10 µs


,employeeid,name,salary,departmentid
0,1,Chief,100,1
1,7,Adm 2,110,4
2,10,Legal 1,110,2
3,12,IT 6,200,3


### 3. List departments that have less than 3 people in it

In [124]:
%time


query = '''
SELECT b.name, COUNT(DISTINCT a.employeeid) as employee_count
FROM employees a
RIGHT JOIN departments b
ON a.departmentid = b.departmentid
GROUP BY a.departmentid, b.name
HAVING COUNT(DISTINCT a.employeeid) < 3
'''

small_departments = psql.read_sql(query, connection)


small_departments.head()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


,name,employee_count
0,Exec,1
1,Legal,2
2,Admin,2
3,Nobody,0


### 4. List all departments along with the number of people there.

**NOTE**: (tricky - people often do an "inner join" leaving out empty departments)

In [148]:
%time


query = '''
SELECT b.name, COUNT(DISTINCT a.employeeid) as employee_count
FROM employees a
RIGHT JOIN departments b
ON a.departmentid = b.departmentid
GROUP BY a.departmentid, b.name
'''

department_employee_count = psql.read_sql(query, connection)


department_employee_count.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


,name,employee_count
0,Exec,1
1,Legal,2
2,IT,7
3,Admin,2
4,Nobody,0


### 5. List employees that don't have a boss in the same department

In [149]:
%time

# table a is a table of boss id's and there respective departments
# table b is a list of employee id's
# WHERE clause selects entries from table of boss id's where the department id in the boss table
# is not present in the full table of employee's and there departmentid's
query = '''
SELECT a.employeeid, a.name
FROM employees a
JOIN employees b
ON a.bossid = b.employeeid
WHERE a.departmentid != b.departmentid
'''

no_boss = psql.read_sql(query, connection)


no_boss.head()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


,employeeid,name
0,2,CTO
1,3,CFO
2,6,Adm 1
3,7,Adm 2
4,9,IT 4


### 6. List all departments along with the total salary there

In [154]:
%time

query = '''
SELECT b.name, SUM(a.salary)
FROM employees a
RIGHT JOIN departments b
ON a.departmentid = b.departmentid
GROUP BY b.name
'''


department_salaries = psql.read_sql(query, connection)


department_salaries.head()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


,name,sum
0,Nobody,NaN
1,Legal,210.0
2,Exec,100.0
3,Admin,130.0
4,IT,665.0
